In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

## Reading the files && Meta Data 


In [2]:
dataframe_train=pd.read_csv("D:\python\kaggle assignment 1\\telecom-churn-case-study-hackathon-c35\\train.csv")
dataframe_test=pd.read_csv("D:\python\kaggle assignment 1\\telecom-churn-case-study-hackathon-c35\\test.csv")

In [3]:
dataframe_train.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,...,0,0,NaN,NaN,NaN,1958,0.0,0.0,0.0,0
1,1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,...,0,0,NaN,1.0,NaN,710,0.0,0.0,0.0,0
2,2,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,60.806,103.176,...,0,0,NaN,NaN,NaN,882,0.0,0.0,0.0,0
3,3,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,156.362,205.260,...,0,0,NaN,NaN,NaN,982,0.0,0.0,0.0,0
4,4,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,240.708,128.191,...,1,0,1.0,1.0,1.0,647,0.0,0.0,0.0,0


it has 172 columns and 69999 rows in the trainng data.

In [4]:
dataframe_train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Data columns (total 172 columns):
 #    Column                    Dtype  
---   ------                    -----  
 0    id                        int64  
 1    circle_id                 int64  
 2    loc_og_t2o_mou            float64
 3    std_og_t2o_mou            float64
 4    loc_ic_t2o_mou            float64
 5    last_date_of_month_6      object 
 6    last_date_of_month_7      object 
 7    last_date_of_month_8      object 
 8    arpu_6                    float64
 9    arpu_7                    float64
 10   arpu_8                    float64
 11   onnet_mou_6               float64
 12   onnet_mou_7               float64
 13   onnet_mou_8               float64
 14   offnet_mou_6              float64
 15   offnet_mou_7              float64
 16   offnet_mou_8              float64
 17   roam_ic_mou_6             float64
 18   roam_ic_mou_7             float64
 19   roam_ic_mou_8             float64
 20   roam

## Exploratory Data Analysis and Feature elimination

In [5]:
dataframe_train["churn_probability"].value_counts()

0    62867
1     7132
Name: churn_probability, dtype: int64

Data is highly imbalanced in nature.

Converting the last date of the month to datetime using to_datetime function

In [6]:
dataframe_train["last_date_of_month_6"]=pd.to_datetime(dataframe_train["last_date_of_month_6"])
dataframe_train["last_date_of_month_7"]=pd.to_datetime(dataframe_train["last_date_of_month_7"])
dataframe_train["last_date_of_month_8"]=pd.to_datetime(dataframe_train["last_date_of_month_8"])

### dropping the last_date_of_month_*6,7,8
it is all the same with no variance hence of no use to us.so we will be dropping these columns.

In [7]:
print(dataframe_train.last_date_of_month_6.dt.day.value_counts())
print(dataframe_train.last_date_of_month_7.dt.day.value_counts())
print(dataframe_train.last_date_of_month_8.dt.day.value_counts())

30    69999
Name: last_date_of_month_6, dtype: int64
31.0    69600
Name: last_date_of_month_7, dtype: int64
31.0    69266
Name: last_date_of_month_8, dtype: int64


In [8]:
#dropping the columns of last_date_of_month_*6,7,8
dataframe_train=dataframe_train.drop(columns=["last_date_of_month_6","last_date_of_month_7","last_date_of_month_8"])
# making a list of dropped columns 
dropped_columns=["last_date_of_month_6","last_date_of_month_7","last_date_of_month_8"]

### Keeping track of the dropped columns in the dropped_columns list 
it is used for keeping the track of dropped columns so that we can drop it in the dataframe train place too.

In [9]:
#using a for loop
continuous_variable_float=dataframe_train.select_dtypes(include="float64").columns
continuous_variable_int=dataframe_train.select_dtypes(include="int64").columns

In [10]:
continuous_variable_int=list(continuous_variable_int)
continuous_variable_int.pop(-1)

'churn_probability'

### bivariate analusis of all the continous integer and float variables 

### checking the standard deviation for the variance present in the variables 
if the standard deviation is equal to 0.that columns will be dropped because it is the same no matter waht hence very hard to get inforation from it. 

In [11]:
dropping_int=[]
for i in continuous_variable_int:
    newframe=dataframe_train[[i,"churn_probability"]]
    print(newframe.groupby(by="churn_probability").mean())
    print("Mean ",round(newframe[i].mean(),2))
    print("Standard Deviation",round(newframe[i].std(),2))
    if newframe[i].std()==0:
        dropping_int.append(i)
#continue to update the dropping the columns 
for j in dropping_int:
    dropped_columns.append(j)

                             id
churn_probability              
0                  35010.356721
1                  34898.893017
Mean  34999.0
Standard Deviation 20207.12
                   circle_id
churn_probability           
0                      109.0
1                      109.0
Mean  109.0
Standard Deviation 0.0
                   total_rech_num_6
churn_probability                  
0                          7.504939
1                          8.109366
Mean  7.57
Standard Deviation 7.04
                   total_rech_num_7
churn_probability                  
0                          7.856173
1                          6.388811
Mean  7.71
Standard Deviation 7.05
                   total_rech_num_8
churn_probability                  
0                          7.629774
1                          3.656338
Mean  7.22
Standard Deviation 7.2
                   total_rech_amt_6
churn_probability                  
0                        325.570888
1                        350.784072

In [12]:
dropping_float=[]
for i in continuous_variable_float:
    newframe=dataframe_train[[i,"churn_probability"]]
    print(newframe.groupby(by="churn_probability").mean())
    print("Mean ",round(newframe[i].mean(),2))
    print("Standard Deviation",round(newframe[i].std(),2))
    if newframe[i].std()==0:
        dropping_float.append(i)
#again continuing the dropped columns
for j in dropping_float:
    dropped_columns.append(j)

                   loc_og_t2o_mou
churn_probability                
0                             0.0
1                             0.0
Mean  0.0
Standard Deviation 0.0
                   std_og_t2o_mou
churn_probability                
0                             0.0
1                             0.0
Mean  0.0
Standard Deviation 0.0
                   loc_ic_t2o_mou
churn_probability                
0                             0.0
1                             0.0
Mean  0.0
Standard Deviation 0.0
                       arpu_6
churn_probability            
0                  280.280668
1                  308.289067
Mean  283.13
Standard Deviation 334.21
                       arpu_7
churn_probability            
0                  284.891073
1                  219.081400
Mean  278.19
Standard Deviation 344.37
                       arpu_8
churn_probability            
0                  297.535694
1                  114.226380
Mean  278.86
Standard Deviation 351.92
                

Mean  21.69
Standard Deviation 87.31
                   std_ic_t2m_mou_8
churn_probability                  
0                         21.859982
1                          9.664736
Mean  21.09
Standard Deviation 81.53
                   std_ic_t2f_mou_6
churn_probability                  
0                          2.239385
1                          1.232947
Mean  2.15
Standard Deviation 16.52
                   std_ic_t2f_mou_7
churn_probability                  
0                          2.319724
1                          0.886415
Mean  2.2
Standard Deviation 16.17
                   std_ic_t2f_mou_8
churn_probability                  
0                          2.188157
1                          0.401007
Mean  2.08
Standard Deviation 15.87
                   std_ic_t2o_mou_6
churn_probability                  
0                               0.0
1                               0.0
Mean  0.0
Standard Deviation 0.0
                   std_ic_t2o_mou_7
churn_probability             

#### fb_user_6 variable analysis :


In [13]:
dataframe_train["fb_user_6"].value_counts()

1.0    16098
0.0     1470
Name: fb_user_6, dtype: int64

In [14]:
dataframe_train["fb_user_6"].isnull().sum()

52431

#### it has too many null values 
so now we remove the extra nulll values

In [15]:
columns=dataframe_train.columns

### Null value percentage 
any varuiable above than 50 percent is dropped 

In [16]:
list1=[]
for i in columns:
    percentage=dataframe_train[i].isnull().sum()/len(dataframe_train)
    if percentage>=0.5:
        list1.append((i,percentage))
nonnull=pd.DataFrame(list1,columns=("variable","percentage_of_nulls"))

In [17]:
for i in nonnull["variable"]:
    dropped_columns.append(i)

In [18]:
#dropping the variables 
dataframe_train.drop(columns=(dropping_int),inplace=True)
dataframe_train.drop(columns=(dropping_float),inplace=True)
dataframe_train.drop(columns=(nonnull["variable"]),inplace=True)

In [19]:
dataframe_train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Data columns (total 129 columns):
 #    Column               Dtype  
---   ------               -----  
 0    id                   int64  
 1    arpu_6               float64
 2    arpu_7               float64
 3    arpu_8               float64
 4    onnet_mou_6          float64
 5    onnet_mou_7          float64
 6    onnet_mou_8          float64
 7    offnet_mou_6         float64
 8    offnet_mou_7         float64
 9    offnet_mou_8         float64
 10   roam_ic_mou_6        float64
 11   roam_ic_mou_7        float64
 12   roam_ic_mou_8        float64
 13   roam_og_mou_6        float64
 14   roam_og_mou_7        float64
 15   roam_og_mou_8        float64
 16   loc_og_t2t_mou_6     float64
 17   loc_og_t2t_mou_7     float64
 18   loc_og_t2t_mou_8     float64
 19   loc_og_t2m_mou_6     float64
 20   loc_og_t2m_mou_7     float64
 21   loc_og_t2m_mou_8     float64
 22   loc_og_t2f_mou_6     float64
 23   loc_og_t2

In [20]:
continuous_variable_float=list(dataframe_train.select_dtypes(include="float64").columns)
continuous_variable_int=list(dataframe_train.select_dtypes(include="int64").columns)
continuous_variable_object=list(dataframe_train.select_dtypes(include="object").columns)
continuous_variable_int.pop(-1)

'churn_probability'

### Mean and Std of the variables :
here in many variables the mean is smaller than standard deviation meansd the value are far wide spread than accumulated bunches

In [21]:
for i in continuous_variable_int:
    newframe=dataframe_train[[i,"churn_probability"]]
    print(newframe.groupby(by="churn_probability").mean())
    print("Mean ",round(newframe[i].mean(),2))
    print("Standard Deviation",round(newframe[i].std(),2))

                             id
churn_probability              
0                  35010.356721
1                  34898.893017
Mean  34999.0
Standard Deviation 20207.12
                   total_rech_num_6
churn_probability                  
0                          7.504939
1                          8.109366
Mean  7.57
Standard Deviation 7.04
                   total_rech_num_7
churn_probability                  
0                          7.856173
1                          6.388811
Mean  7.71
Standard Deviation 7.05
                   total_rech_num_8
churn_probability                  
0                          7.629774
1                          3.656338
Mean  7.22
Standard Deviation 7.2
                   total_rech_amt_6
churn_probability                  
0                        325.570888
1                        350.784072
Mean  328.14
Standard Deviation 404.21
                   total_rech_amt_7
churn_probability                  
0                        331.234352
1  

In [22]:
for i in continuous_variable_float:
    newframe=dataframe_train[[i,"churn_probability"]]
    print(newframe.groupby(by="churn_probability").mean())
    print("Mean ",round(newframe[i].mean(),2))
    print("Standard Deviation",round(newframe[i].std(),2))
    print("The total number of null-values",newframe[i].isnull().sum())

                       arpu_6
churn_probability            
0                  280.280668
1                  308.289067
Mean  283.13
Standard Deviation 334.21
The total number of null-values 0
                       arpu_7
churn_probability            
0                  284.891073
1                  219.081400
Mean  278.19
Standard Deviation 344.37
The total number of null-values 0
                       arpu_8
churn_probability            
0                  297.535694
1                  114.226380
Mean  278.86
Standard Deviation 351.92
The total number of null-values 0
                   onnet_mou_6
churn_probability             
0                   127.231453
1                   191.239495
Mean  133.15
Standard Deviation 299.96
The total number of null-values 2768
                   onnet_mou_7
churn_probability             
0                   132.755677
1                   146.320051
Mean  133.89
Standard Deviation 311.28
The total number of null-values 2687
                   on

The total number of null-values 3703
                   std_ic_t2f_mou_6
churn_probability                  
0                          2.239385
1                          1.232947
Mean  2.15
Standard Deviation 16.52
The total number of null-values 2768
                   std_ic_t2f_mou_7
churn_probability                  
0                          2.319724
1                          0.886415
Mean  2.2
Standard Deviation 16.17
The total number of null-values 2687
                   std_ic_t2f_mou_8
churn_probability                  
0                          2.188157
1                          0.401007
Mean  2.08
Standard Deviation 15.87
The total number of null-values 3703
                   std_ic_mou_6
churn_probability              
0                     31.816912
1                     37.693897
Mean  32.36
Standard Deviation 104.38
The total number of null-values 2768
                   std_ic_mou_7
churn_probability              
0                     34.238765
1             

In [23]:
for i in continuous_variable_object:
    newframe=dataframe_train[[i,"churn_probability"]]
    print(newframe.groupby(by=i).sum())
    print("total number of null values",newframe[i].isnull().sum())
    print(newframe["churn_probability"].value_counts())

                     churn_probability
date_of_last_rech_6                   
6/1/2014                            22
6/10/2014                           41
6/11/2014                           68
6/12/2014                          171
6/13/2014                          123
6/14/2014                          279
6/15/2014                           85
6/16/2014                           84
6/17/2014                          303
6/18/2014                          158
6/19/2014                          137
6/2/2014                            16
6/20/2014                          130
6/21/2014                          305
6/22/2014                          140
6/23/2014                          162
6/24/2014                          311
6/25/2014                          339
6/26/2014                          375
6/27/2014                          712
6/28/2014                          582
6/29/2014                          942
6/3/2014                            49
6/30/2014                

In [24]:
#dropping the object variables
dataframe_train.drop(columns=continuous_variable_object,inplace=True)
#continuing to update the dropped columns
for j in continuous_variable_object:
    dropped_columns.append(j)

In [25]:
dataframe_train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Data columns (total 126 columns):
 #    Column              Dtype  
---   ------              -----  
 0    id                  int64  
 1    arpu_6              float64
 2    arpu_7              float64
 3    arpu_8              float64
 4    onnet_mou_6         float64
 5    onnet_mou_7         float64
 6    onnet_mou_8         float64
 7    offnet_mou_6        float64
 8    offnet_mou_7        float64
 9    offnet_mou_8        float64
 10   roam_ic_mou_6       float64
 11   roam_ic_mou_7       float64
 12   roam_ic_mou_8       float64
 13   roam_og_mou_6       float64
 14   roam_og_mou_7       float64
 15   roam_og_mou_8       float64
 16   loc_og_t2t_mou_6    float64
 17   loc_og_t2t_mou_7    float64
 18   loc_og_t2t_mou_8    float64
 19   loc_og_t2m_mou_6    float64
 20   loc_og_t2m_mou_7    float64
 21   loc_og_t2m_mou_8    float64
 22   loc_og_t2f_mou_6    float64
 23   loc_og_t2f_mou_7    float64
 24   

In [26]:
list1=[]
for i in dataframe_train.columns:
    list1.append((i,dataframe_train[i].isnull().sum()))
nullvalues=pd.DataFrame(list1,columns=["Variables","Null Values"])

In [27]:
imputing_values=nullvalues[nullvalues["Null Values"]!=0]

In [28]:
#imputing the mean values
for i in imputing_values["Variables"]:
    dataframe_train[i].fillna(dataframe_train[i].mean(),inplace=True)

In [29]:
#sns.pairplot(dataframe_train)

In [30]:
#for i in dataframe_train.columns:
#    sns.displot(dataframe_train[i])

## Train and test split

In [31]:
columns=list(dataframe_train.columns)
columns.remove("id")
columns.remove("churn_probability")
y=dataframe_train["churn_probability"]
X=dataframe_train[columns]

In [32]:
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8,random_state=1)

## Model Selection

## Logistic Regression

In [56]:
minmx=MinMaxScaler()
X_train_min=minmx.fit_transform(X_train)
X_test_min=minmx.transform(X_test)
lr=LogisticRegression(max_iter=1000)
%time lr.fit(X_train_min,y_train)
y_pred_train=lr.predict(X_train_min)
y_pred_test=lr.predict(X_test_min)

Wall time: 1.62 s


In [57]:
confusion_matrix(y_train,y_pred_train)

array([[50026,   269],
       [ 4721,   983]], dtype=int64)

In [58]:
roc_auc_score(y_train,y_pred_train)

0.5834933795933586

In [59]:
print(confusion_matrix(y_test,y_pred_test))
print(roc_auc_score(y_test,y_pred_test))
print(precision_score(y_test,y_pred_test))
print(recall_score(y_test,y_pred_test))

[[12513    59]
 [ 1215   213]]
0.5722333476820571
0.7830882352941176
0.14915966386554622


In [60]:
print(confusion_matrix(y_train,y_pred_train))
print(roc_auc_score(y_train,y_pred_train))
print(precision_score(y_train,y_pred_train))
print(recall_score(y_train,y_pred_train))

[[50026   269]
 [ 4721   983]]
0.5834933795933586
0.7851437699680511
0.1723352033660589


## Using PCA 

### finding out the n_components 

In [38]:
#it is important to normalize the variables
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
pca=PCA()
pca.fit(X_train)
array=np.cumsum(pca.explained_variance_ratio_)
# keeping the 95 percent of variance
components=0
for i in array:
    components+=1
    if i>=0.97:
        break
print("the components that will preserve the 97 percent of the variance",components)

the components that will preserve the 97 percent of the variance 76


In [39]:
array

array([0.13100527, 0.20748118, 0.25963174, 0.30517332, 0.34151057,
       0.374154  , 0.40315217, 0.43085099, 0.45554838, 0.47932469,
       0.50213424, 0.52464547, 0.54673077, 0.56825659, 0.58848263,
       0.60609227, 0.62269793, 0.63696479, 0.65086974, 0.66457093,
       0.67798445, 0.69112627, 0.70388131, 0.71567024, 0.72631639,
       0.73643359, 0.74639088, 0.75613693, 0.76488782, 0.77330976,
       0.78135618, 0.78930907, 0.79703201, 0.80462498, 0.81190586,
       0.81889369, 0.82575407, 0.83252208, 0.83903857, 0.84533452,
       0.85120642, 0.85702483, 0.8624984 , 0.86780602, 0.8726203 ,
       0.87716245, 0.88167821, 0.88612362, 0.8903486 , 0.89446684,
       0.89853989, 0.90252325, 0.90642568, 0.91014569, 0.91372   ,
       0.91722333, 0.9206484 , 0.92403979, 0.92726524, 0.93027626,
       0.93321572, 0.93609271, 0.93893299, 0.94170831, 0.94444403,
       0.94709176, 0.94972327, 0.95231271, 0.95484706, 0.95725657,
       0.95960683, 0.96192788, 0.96420858, 0.96640784, 0.96854

In [40]:
pca=PCA(n_components=76)
%time pca.fit(X_train)


Wall time: 1.15 s


PCA(n_components=76)

In [41]:
X_train_pca=pca.transform(X_train)
X_test_pca=pca.transform(X_test)

In [42]:
lr_pca=LogisticRegression(max_iter=500)
%time lr_pca.fit(X_train_pca,y_train)
y_pred_train=lr_pca.predict(X_train_pca)
y_pred_test=lr_pca.predict(X_test_pca)

Wall time: 514 ms


In [43]:
print("The train part")
print(confusion_matrix(y_train,y_pred_train))
print("the Area under the curve is ",roc_auc_score(y_train,y_pred_train))
print("the precision score for the tarin is ",precision_score(y_train,y_pred_train))
print("the recall score for the train is ",recall_score(y_train,y_pred_train))

The train part
[[49777   518]
 [ 4693  1011]]
the Area under the curve is  0.5834724023771669
the precision score for the tarin is  0.6612164813603663
the recall score for the train is  0.17724403927068724


In [44]:
print("The Test part")
print(confusion_matrix(y_test,y_pred_test))
print("The area under the curve is ",roc_auc_score(y_test,y_pred_test))
print("the precision score is",precision_score(y_test,y_pred_test))
print("the recall score is ",recall_score(y_test,y_pred_test))

The Test part
[[12468   104]
 [ 1204   224]]
The area under the curve is  0.574295196920639
the precision score is 0.6829268292682927
the recall score is  0.1568627450980392


## Decision trees


In [45]:
tree=DecisionTreeClassifier(max_depth=10,min_samples_leaf=20)
%time tree.fit(X_train,y_train)
y_pred_train=tree.predict(X_train)
y_pred_test=tree.predict(X_test)

Wall time: 3.19 s


In [46]:
print("The train part")
print(confusion_matrix(y_train,y_pred_train))
print("the Area under the curve is ",roc_auc_score(y_train,y_pred_train))
print("the precision score for the tarin is ",precision_score(y_train,y_pred_train))
print("the recall score for the train is ",recall_score(y_train,y_pred_train))

The train part
[[49349   946]
 [ 1963  3741]]
the Area under the curve is  0.8185232566148645
the precision score for the tarin is  0.7981651376146789
the recall score for the train is  0.6558555399719496


In [47]:
print("The Test part")
print(confusion_matrix(y_test,y_pred_test))
print("The area under the curve is ",roc_auc_score(y_test,y_pred_test))
print("the precision score is",precision_score(y_test,y_pred_test))
print("the recall score is ",recall_score(y_test,y_pred_test))

The Test part
[[12247   325]
 [  586   842]]
The area under the curve is  0.7818923783321793
the precision score is 0.7215081405312768
the recall score is  0.5896358543417367


## Using PCA

In [48]:
tree_pca=DecisionTreeClassifier(max_depth=10,min_samples_leaf=20)
%time tree_pca.fit(X_train_pca,y_train)
y_pred_train=tree_pca.predict(X_train_pca)
y_pred_test=tree_pca.predict(X_test_pca)

Wall time: 4.37 s


In [49]:
print("The train part")
print(confusion_matrix(y_train,y_pred_train))
print("the Area under the curve is ",roc_auc_score(y_train,y_pred_train))
print("the precision score for the tarin is ",precision_score(y_train,y_pred_train))
print("the recall score for the train is ",recall_score(y_train,y_pred_train))
print("the accuracy score for the train is :",accuracy_score(y_train,y_pred_train))

The train part
[[49280  1015]
 [ 2934  2770]]
the Area under the curve is  0.7327215954619498
the precision score for the tarin is  0.7318361955085865
the recall score for the train is  0.4856241234221599


In [50]:
print("The Test part")
print(confusion_matrix(y_test,y_pred_test))
print("The area under the curve is ",roc_auc_score(y_test,y_pred_test))
print("the precision score is",precision_score(y_test,y_pred_test))
print("the recall score is ",recall_score(y_test,y_pred_test))

The Test part
[[12173   399]
 [  920   508]]
The area under the curve is  0.6620025515774238
the precision score is 0.5600882028665931
the recall score is  0.3557422969187675


## MOdel selection and hyperparameter tuning

In [51]:
pipe=Pipeline([("classifier",LogisticRegression())])
grid_param=[
             {"classifier":[DecisionTreeClassifier()],
              "classifier__max_depth":[2,4,8,10,12],
              "classifier__min_samples_leaf":[10,15,20,25,30]
             },
             {"classifier":[RandomForestClassifier()],
              "classifier__n_estimators":[10,50,100,150,200],
              "classifier__min_samples_leaf":[10,20,30,40,40,100,200]}
    
                                                                      ]
gridsearch=GridSearchCV(pipe,grid_param,cv=5,verbose=0,n_jobs=-1)
best_model=gridsearch.fit(X_train,y_train)

In [52]:
print(best_model.best_estimator_)

Pipeline(steps=[('classifier', RandomForestClassifier(min_samples_leaf=10))])


## Building a random classifier model

In [61]:
rf=RandomForestClassifier(n_estimators=200,min_samples_leaf=10)
%time rf.fit(X_train,y_train)
y_pred_train=rf.predict(X_train)
y_pred_test=rf.predict(X_test)

Wall time: 1min 7s


In [62]:
print("The train part")
print(confusion_matrix(y_train,y_pred_train))
print("the Area under the curve is ",roc_auc_score(y_train,y_pred_train))
print("the precision score for the tarin is ",precision_score(y_train,y_pred_train))
print("the recall score for the train is ",recall_score(y_train,y_pred_train))

The train part
[[49652   643]
 [ 2048  3656]]
the Area under the curve is  0.8140845728295623
the precision score for the tarin is  0.8504303326354966
the recall score for the train is  0.6409537166900421


In [63]:
print("The Test part")
print(confusion_matrix(y_test,y_pred_test))
print("The area under the curve is ",roc_auc_score(y_test,y_pred_test))
print("the precision score is",precision_score(y_test,y_pred_test))
print("the recall score is ",recall_score(y_test,y_pred_test))
print("the accuracy of the model is",accuracy_score(y_test,y_pred_test))

The Test part
[[12314   258]
 [  556   872]]
The area under the curve is  0.7950612316195966
the precision score is 0.7716814159292036
the recall score is  0.6106442577030813
the accuracy of the model is 0.9418571428571428


In [74]:
dataframe_test.drop(columns=dropped_columns,inplace=True)

In [79]:
id_columns=list(dataframe_test.columns)
id_columns.remove("id")
newdata=dataframe_test[id_columns]

In [81]:
newdata.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 124 columns):
 #    Column              Dtype  
---   ------              -----  
 0    arpu_6              float64
 1    arpu_7              float64
 2    arpu_8              float64
 3    onnet_mou_6         float64
 4    onnet_mou_7         float64
 5    onnet_mou_8         float64
 6    offnet_mou_6        float64
 7    offnet_mou_7        float64
 8    offnet_mou_8        float64
 9    roam_ic_mou_6       float64
 10   roam_ic_mou_7       float64
 11   roam_ic_mou_8       float64
 12   roam_og_mou_6       float64
 13   roam_og_mou_7       float64
 14   roam_og_mou_8       float64
 15   loc_og_t2t_mou_6    float64
 16   loc_og_t2t_mou_7    float64
 17   loc_og_t2t_mou_8    float64
 18   loc_og_t2m_mou_6    float64
 19   loc_og_t2m_mou_7    float64
 20   loc_og_t2m_mou_8    float64
 21   loc_og_t2f_mou_6    float64
 22   loc_og_t2f_mou_7    float64
 23   loc_og_t2f_mou_8    float64
 24   

In [82]:
#checking for non-null values
list1=[]
for i in newdata.columns:
    list1.append((i,newdata[i].isnull().sum()))
nonnull=pd.DataFrame(list1,columns=("variable","nonnull"))

In [86]:
pd.set_option("display.max_columns",50,"display.max_rows",150)

In [87]:
nonnull

,variable,nonnull
0,arpu_6,0
1,arpu_7,0
2,arpu_8,0
3,onnet_mou_6,1169
4,onnet_mou_7,1172
5,onnet_mou_8,1675
6,offnet_mou_6,1169
7,offnet_mou_7,1172
8,offnet_mou_8,1675
9,roam_ic_mou_6,1169


In [90]:
impute_columns=nonnull[nonnull["nonnull"]!=0]

In [91]:
impute_columns

,variable,nonnull
3,onnet_mou_6,1169
4,onnet_mou_7,1172
5,onnet_mou_8,1675
6,offnet_mou_6,1169
7,offnet_mou_7,1172
8,offnet_mou_8,1675
9,roam_ic_mou_6,1169
10,roam_ic_mou_7,1172
11,roam_ic_mou_8,1675
12,roam_og_mou_6,1169


In [94]:
for i in impute_columns["variable"]:
    newdata[i].fillna(newdata[i].mean(),inplace=True)

C:\Users\shray\anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [95]:
y_pred_main=rf.predict(newdata)

In [106]:
s1=dataframe_test["id"]
s2=pd.Series(y_pred_main,name="churn_probability")
final=pd.concat([s1,s2],axis=1)
final.head()

,id,churn_probability
0,69999,0
1,70000,0
2,70001,0
3,70002,0
4,70003,0


In [107]:
final.to_csv("D:\python\kaggle assignment 1\\telecom-churn-case-study-hackathon-c35\submission.csv",index=False)

In [116]:
final.shape


(30000, 2)